# Базы данных

В **Python** определен стандарт *API* баз данных ([PEP 249](https://www.python.org/dev/peps/pep-0249/)), на основе которого построены *API* для различных баз данных, в частности **MySQL** и **PostgreSQL**.

Для работы с **MySQL** требуется установка модуля **mysql-connector-python**, для **PostgreSQL** - модуля **psycopg2**. Примеры приведены на основе MySQL, но за счет универсальности API применимы и для **PostgreSQL**.

**В тексте блокнота указаны вымышленные параметры подключения к базе данных (БД). Код в этом блокноте нельзя запускать, следует прочитать код и посмотреть полученный ранее вывод. Если вы запустили ячейку, то удалите ваш блокнот и скачайте его заново.**

```
import mysql.connector
from mysql.connector import Error

try:
    conn = mysql.connector.connect(
        host='111.111.111.111', port=3333, 
        user='user', password='txavChRizHiL;3GW7y8cw6tn',
        database='python_mysql'
    )
    print('Connected to MySQL database')
except Error as e:
    print(e)
```
Connected to MySQL database

Создаем объект курсора для обращения к БД:

```
cursor = conn.cursor()

# Execute SQL requests
try:
    # Пытаемся удалить таблицу. 
    cursor.execute("DROP TABLE customers")
except:
    # Если ее не существует - игнорируем ошибку
    pass

cursor.execute("CREATE TABLE customers (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(255), address VARCHAR(255))")  
# В PostgreSQL вместо AUTO_INCREMENT необходимо написать SERIAL

# Вставка одной строки
# В бд mysql используется "%s"
cursor.execute("INSERT INTO customers (name, address) VALUES (%s, %s)", ('test', 'bla-bla'))
conn.commit()

# Вставка нескольких строк
cursor.executemany(
    "INSERT INTO customers (name, address) VALUES (%s, %s)", 
    [
        ('test2', '12345'),
        ('test3', '98765'),
    ]
)
print('Last inserted id:', cursor.lastrowid)
conn.commit()
```
Last inserted id: 2

```
cursor.execute("SELECT * FROM customers")

row = cursor.fetchone()  # Получаем 1 запись

while row:  
    print(row)  
    row = cursor.fetchone()
```
(1, 'test', 'bla-bla')<br>
(2, 'test2', '12345')<br>
(3, 'test3', '98765')

```
cursor.execute("SELECT * FROM customers")

# Получаем все записи  
rows = cursor.fetchall()
print(rows)
```
[(1, 'test', 'bla-bla'), (2, 'test2', '12345'), (3, 'test3', '98765')]

```
def iter_rows(cursor, chunk_size):
    while True:
        # Получаем chunk_size записей за раз
        rows = cursor.fetchmany(chunk_size)
        yield from rows
        if len(rows) < chunk_size:
            # Больше записей нет
            break

cursor.execute("SELECT * FROM customers")

rows = list(iter_rows(cursor, 2))
print(rows)
```
[(1, 'test', 'bla-bla'), (2, 'test2', '12345'), (3, 'test3', '98765')]

```
# Изменяем запись  
cursor.execute("UPDATE customers SET name=%s WHERE id=%s", ('*EDIT*', 2))
conn.commit()

# Получаем все записи
cursor.execute("SELECT * FROM customers")
rows = cursor.fetchall()
print(rows)

# Удаляем запись  
cursor.execute("DELETE FROM customers WHERE id=%s", (1,))
conn.commit()

# Получаем все записи
cursor.execute("SELECT * FROM customers")
rows = cursor.fetchall()
print(rows)
    
cursor.close()
conn.close()
```
[(1, 'test', 'bla-bla'), (2, '\*EDIT\*', '12345'), (3, 'test3', '98765')]<br>
[(2, '\*EDIT\*', '12345'), (3, 'test3', '98765')]

# Основные команды

* `con.close()` – закрывает соединение с сервером базы данных.
* `con.commit()` – подтверждает все незавершенные транзакции
* `con.rollback()` – откатывает все изменения в базе данных до момента, когда были запущены незавершённые транзакции.
* `con.cursor()` – создаёт новый *курсор*. Это экземпляр класса **Cursor**. *Курсор* – это объект, который используется для **SQL** запросов и получения результата.
* `con.autocommit()` – используется для автоматического завершения транзакции.
* `con.affected_rows()` – возвращает число строк, на которые повлиял последний запрос. Лучше использовать `cur.rowcount()`. (не стандартный)
* `con.errno()` – возвращает номер ошибки, если она произошла.
* `con.error()` – возвращает описание ошибки, если она произошла.

## Команды курсора

* `cur.close()` – закрывает *курсор*, предотвращая выполнение каких-либо запросов с его помощью.
* `cur.callproc(procname [, param])` – вызывает хранимую процедуру.
* `cur.execute(query [, param])` – выполняет запрос к базе данных (query).
* `cur.executemany(query [, paramsequence])` – многократное выполнение запросов к базе данных (query).
* `cur.fetchone()` – возвращает следующую запись из набора данных полученного вызовом cur.execute*()
* `cur.fetchmany([size])` – возвращает последовательность записей из набора данных.
* `cur.fetchall()` – возвращает последовательность всех записей оставшихся в полученном наборе данных.
* `cur.nextset()` – пропускает все оставшиеся записи в текущем наборе данных и переходит к следующему набору.
* `cur.description` – возвращает последовательность кортежей с информацией о каждом столбце в текущем наборе данных. Кортеж имеет вид (*name*, *type_code*, *display_size*, *internal_size*, *precision*, *scale*, *null_ok*)
* `cur.arraysize` – целое число, которое используется методом *cur.fetchmany* в качестве значения по умолчанию.
* `cur.rowcount` – возвращает число строк, на которые повлиял последний запрос.
